In [ ]:
import csv
from google.colab import drive
drive.mount('/content/drive')

store ids of non-rejected CVE instances

In [ ]:
#change path of NVD_2000-2021.csv
count = 0
with open('/content/drive/MyDrive/Research/NVD_2000-2021.csv', 'r',encoding='utf-8') as f:
    file = csv.DictReader(f)
    for i in file:
        count+=1
print(count)

In [ ]:
not_rejected_ids = {} 
with open('/content/labelling.csv', 'r',encoding='utf-8') as f:
    labelling = csv.DictReader(f)
    for row in labelling:
        not_rejected_ids[row['CVE-id']] = int(row['Dominant_Topic'])

len(not_rejected_ids)


In [ ]:
import pandas as pd
df = pd.read_csv("/content/labelling.csv")
print(df[df['CVE-id']=='CVE-2020-10739']['Dominant_Topic'])

obtain date of creation for non-rejected CVE instances (stored in not_rejected_dates)

In [ ]:
count = 0
not_rejected_dates = {}
with open('/content/drive/MyDrive/Research/NVD_2000-2021.csv', 'r',encoding='utf-8') as ff:
    all_cves = csv.DictReader(ff)
    for row in all_cves:
        id = row['vulnerability_id']
        if id in not_rejected_ids:
            count = count + 1
            year = row['reporting_time'].split("/")[2]
            month = row['reporting_time'].split("/")[0]
            not_rejected_dates[id] = []
            not_rejected_dates[id].append(year)
            not_rejected_dates[id].append(month)
            not_rejected_dates[id].append(not_rejected_ids[id]) #domtop
            # print(count)
                               
print(len(not_rejected_dates))
# for k,v in not_rejected_dates.items():
#     print(k,v[0],v[1])

In [ ]:
#store the year information for non-rejected cve instances in csv file
with open('/content/year-info.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['cve', 'Year','Month','DomTop'])
    for k,v in not_rejected_dates.items():
        writer.writerow([k,v[0],v[1],v[2]])

get total cves per year

In [ ]:

with open('/content/year-info.csv', 'r', newline='') as file:
    reader = csv.DictReader(file)
    for r in reader:
        yr = r['Year']
        if yr in year_dict:
          year_dict[yr]+=1
        else:
          year_dict[yr]=1


# for k,v in year_dict.items():
#   print(k,v

get cves per month BY TOPIC

In [ ]:
months_dict={} #{2000:[i=0 jan, i=1 feb....],2001:[....],....}
desired_topic = 17
with open('/content/year-info.csv', 'r', newline='') as file:
    reader = csv.DictReader(file)
    for r in reader:
        yr = r['Year']
        cveid = r['cve']
        month = r['Month']
        domTop = int(r['DomTop'])
        i = int(month)-1
        if domTop==desired_topic:
          if yr in months_dict:
            months_dict[yr][i]+=1
          else:
            temp = [0]*12
            temp[i]=1
            months_dict[yr]=temp
        else:
          continue

print("For topic: ", desired_topic)
header = []
monthsData = []
header.append(desired_topic)
monthsData.append("")
for k,v in months_dict.items():
  yrabbr = k[-2:]
  for i in range(1,13):
    h = "'"+str(i)+"/"+yrabbr
    header.append(h)
  
  for i in v:
    monthsData.append(i)
print(header, len(header))
print(monthsData,len(monthsData))

with open('/content/months.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    # writer.writerow(header)
    writer.writerow(monthsData)

analyze topic distribution trends

In [ ]:
def find_max_key(dict):
    max_v = 0
    max_k = ""
    for k,v in dict.items():
        if k == 'CVE-ID':
            continue
        if k == 'Year':
            continue
        if k == 'Month':
            continue
        if float(v)>max_v:
            max_v = float(v)
            max_k = k
            
    return max_k
            
        
#find trend where each vulnerbality is described using most dominant topic
count = 0
results = {"2011":{},"2012":{}, "2013":{},"2014":{},"2015":{},"2016":{},"2017":{},"2018":{},"2019":{},"2020":{},"2021":{}}
desired_topic = "Topic_17"
with open('importance.csv', 'r', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if find_max_key(row)==desired_topic:
            year = row['Year']
            month = row['Month']
            if month in results[year]:
                results[year][month]+=1
            else:
                results[year][month]=1
#         if row['Year']==desired_year:
#             dominantTopic = find_max_key(row)
#             if dominantTopic in topic_count:
#                 topic_count[dominantTopic]+=1
#             else:
#                 topic_count[dominantTopic] = 1

# for k,v in results.items():
#     print(k)
#     for m,c in v.items():
#         print(m,c)
with open('trend-analysis-results.csv', 'a', newline='') as output:
    writer = csv.writer(output)
    writer.writerow([desired_topic])
    temp = []
    for k,v in results.items():
        for m,c in v.items():
            temp.append(c)
    writer.writerow(temp)

In [ ]:
months = ["Jan", "Feb","Mar","Apr","May","June","July","Aug","Sept","Oct","Nov","Dec"]
temp = []
for y in range(11,22):
    for m in months:
        entry = " "
        entry+=m
        entry+="-"
        entry+=str(y)
        temp.append(entry)
    with open('temp.csv', 'a', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(temp)

analyze topic importance evolution

In [ ]:
topic_importances = {"Topic_0": {}, "Topic_1": {},"Topic_2": {},"Topic_3": {},"Topic_4": {},"Topic_5": {},"Topic_6": {},
    "Topic_7": {},
    "Topic_8": {},
    "Topic_9": {},"Topic_10": {},
    "Topic_11": {},
    "Topic_12": {},
    "Topic_13": {},"Topic_14": {},"Topic_15": {},"Topic_16": {},"Topic_17": {}}
count = 0
desired_year = '2000'
# desired_month = '1'

for topic in topic_importances:
    print(topic)
    for desired_month in range(1,13): 
        importance = 0
        total_cves = 0
        with open('importance.csv', 'r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:


                if row['Year']==desired_year and row['Month']==str(desired_month):
                    importance += float(row[topic])
                    total_cves+=1
        topic_importances[topic][str(desired_month)]=importance/total_cves
        # print(topic, importance, total_cves, importance/total_cves)


In [ ]:
for k,v in topic_importances.items():
  print(k,v)
# with open('trend-analysis-results.csv', 'a', newline='') as output:
#     writer = csv.writer(output)
#     writer.writerow([desired_year, desired_month])
#     for k,v in topic_importances.items():
#       print(k,v)
#         # writer.writerow([k,v])

In [ ]:
topics = ["Topic_0", "Topic_1", "Topic_2", "Topic_3", "Topic_4","Topic_5","Topic_6",
    "Topic_7",
    "Topic_8",
    "Topic_9","Topic_10",
    "Topic_11",
    "Topic_12",
    "Topic_13","Topic_14","Topic_15","Topic_16","Topic_17"]
topic_importances = {}
count = 0
# desired_year = '2000'
# desired_month = '1'
results = {}
for topic in topics:
    print(topic)
    for desired_year in range(2000,2022): 
        print(desired_year)
        importance = 0
        total_cves = 0
        with open('importance.csv', 'r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if int(row['Year'])==desired_year:
                    importance += float(row[topic])
                    total_cves+=1
        topic_importances[topic]=importance/total_cves
        # print(desired_year, topic, importance, total_cves, importance/total_cves)
        if topic in results:
          results[topic].append([desired_year, importance/total_cves]) 
        else:
          results[topic] = [desired_year, importance/total_cves]
        
        # print(results)


In [ ]:
# for k,v in results.items():
#   print(k)
#   for i in v:
#     print(str(i)[1:-1])

desired_year = '2021'
# for k,v in topic_importances.items():
#     print(k)
#     temp=[]
#     for m,c in v.items():
# #         print(m,c)
#         temp.append(c)
#     print(temp)
with open('trend-analysis-results.csv', 'w', newline='') as output:
    writer = csv.writer(output)
    for k,v in results.items():
        writer.writerow([k])
        for i in v:
          if type(i) is list:
            print("yes")
            writer.writerow([i[0],i[1]])
          else:
            writer.writerow([i])
          # writer.writerow([i])
        # writer.writerow(temp)

cvss score analysis by category

In [ ]:
original_cve_score={}
with open('NVD_2000-2021.csv', 'r', newline='',encoding='utf-8') as nvd:
    reader = csv.DictReader(nvd)
    for row in reader:
        original_cve_score[row['vulnerability_id']] = row['CVSS-Score']

In [ ]:
cvss_scores={'low':0,'medium':0,'high':0,'critical':0}
desired_topic=17
count = 0
with open('labelling.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if int(row['Dominant_Topic']) == desired_topic:
          score=original_cve_score[row['CVE-id']]
        else: 
          # continue
          score=original_cve_score[row['CVE-id']]

        if score!='N/A' and score!='N/A</a' and len(score)!=0:
            count+=1
            if float(score)>=0.0 and float(score)<=3.9:
                cvss_scores['low']+=1
            elif float(score)>=4.0 and float(score)<=6.9:
                cvss_scores['medium']+=1
            elif float(score)>=7.0 and float(score)<=8.9:
                cvss_scores['high']+=1
            elif float(score)>=9.0 and float(score)<=10.0:
                cvss_scores['critical']+=1
print(count)
# header = [str(desired_topic), 'low', 'med', 'high']
# vals = ['',0,0,0,0]
# print(vals)

# for k,v in cvss_scores.items():
#   if k =='low':
#     vals[1]+=v
#   elif k=='medium':
#     vals[2]+=v
#   elif k == 'high':
#     vals[3]+=v
#   elif k == 'critical':
#     vals[4]+=v

# print(vals)
# with open('cvss.csv', 'a', newline='',encoding='utf-8') as file:
#     writer = csv.writer(file)
#     # writer.writerow(header)
#     writer.writerow(vals)

In [ ]:
cvss_scores_by_year={'low':0,'medium':0,'high':0,'critical':0}
desired_year = '2021'
with open('year-info.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['Year']==desired_year:
            score=original_cve_score[row['cve']]
            if score!='N/A' and score!='N/A</a' and len(score)!=0:
                if float(score)>=0.1 and float(score)<=3.9:
                    cvss_scores_by_year['low']+=1
                elif float(score)>=4.0 and float(score)<=6.9:
                    cvss_scores_by_year['medium']+=1
                elif float(score)>=7.0 and float(score)<=8.9:
                    cvss_scores_by_year['high']+=1
                elif float(score)>=9.0 and float(score)<=10.0:
                    cvss_scores_by_year['critical']+=1

header = [str(2000), 'low', 'med', 'high', 'critical']
vals = ['',0,0,0,0]

for k,v in cvss_scores_by_year.items():
  if k =='low':
    vals[1]+=v
  elif k=='medium':
    vals[2]+=v
  elif k == 'high':
    vals[3]+=v
  elif k == 'critical':
    vals[4]+=v

with open('cvss_year.csv', 'a', newline='',encoding='utf-8') as file:
    writer = csv.writer(file)
    # writer.writerow(header)
    writer.writerow(vals)
    # for k,v in cvss_scores_by_year.items():
    #     writer.writerow([k,v])
    #     print(k,v)

In [ ]:
#get average CVSS scores per topic
all_scores = {}
with open('labelling.csv', 'r', newline='',encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        topic = row['Dominant_Topic']
        score = score=original_cve_score[row['CVE-id']]
        if score!='N/A' and score!='N/A</a' and len(score)!=0:
            if float(score)>=7.0 and float(score)<=8.9: #only high severity
                if topic in all_scores:
                    all_scores[topic].append(float(score))
                else:
                    temp = []
                    temp.append(score)
                    all_scores[topic]=temp
                    
for k,v in all_scores.items():
    sum = 0
    for s in v:
        sum+=float(s)
    print(k,sum/len(v), len(v))